In [9]:
import json
from pathlib import Path

import pandas as pd

In [10]:
filepaths = list(Path(".").glob("*-auto.json"))
dfs = [pd.read_json(filepath) for filepath in filepaths]
dfs[0]

,id,context,question,generation,n_hops
0,2hop__71611_90450,"# Corn production in the United States\nIowa, ...",Who was president when the place where the maj...,Thought:\nIowa is the largest producer of corn...,2
1,2hop__612535_47295,# Thomas H. Makiyama\nThomas H. Makiyama (1928...,What is the area code for the state where Thom...,Thought:\nThomas H. Makiyama was born in Hawai...,2
2,2hop__281903_421645,# The Night We Called It a Day (film)\nThe Nig...,Who is the spouse of the performer of Mistleto...,"Thought:\nFrank Sinatra performed ""Mistletoe a...",2
3,2hop__497910_351187,# Bernalillo County Metropolitan Court\nThe Be...,What county is the city where Kristen Graczyk ...,Thought:\nKristen Graczyk was born in Albuquer...,2
4,2hop__777932_77346,# Clyde Werner\nClyde Werner (born December 10...,The quarterback for the team Clyde Werner play...,Thought:\nClyde Werner played for the Kansas C...,2
...,...,...,...,...,...
163,4hop3__695568_769559_129926_718885,"# 11/6/00 – Seattle, Washington\n11/6/00 – Sea...",What is the country of origin of the album cal...,"Thought:\nThe album 11/6/00 is by Pearl Jam, f...",4
164,4hop1__314513_106790_158589_81091,"# Syria\nSyria (Arabic: سوريا ‎ Sūriyā), offic...",In which part of the world is one source of fo...,Thought:\nThe Cardigans are from Sweden. One s...,4
165,4hop2__724536_733536_75897_36009,"# Culture of Aruba\nThe culture of Aruba, one ...",When was the SNES released in the colonial hol...,"Thought:\nAruba is located in the Caribbean, w...",4
166,4hop1__106947_221169_833580_61459,# Queen Elisabeth Medical Foundation\nThe Quee...,When did Italy fight in the same was as did Al...,"Thought:\nAlbert I was the King of Belgium, wh...",4


In [11]:
ids_list = [df['id'].values for df in dfs]

In [12]:
common_ids = set(ids_list[0]).intersection(*ids_list[1:])
len(common_ids)

123

In [13]:
from itertools import zip_longest

def interweave(*args):
    result = []
    for items in zip_longest(*args, fillvalue=None):
        result.extend(item for item in items if item is not None)
    return result


In [14]:
# test interweave function
# cover happy path and edge cases
assert interweave([1, 2, 3], [4, 5, 6]) == [1, 4, 2, 5, 3, 6]
assert interweave([1, 2, 3], [4, 5]) == [1, 4, 2, 5, 3]
assert interweave([1, 2], [4, 5, 6]) == [1, 4, 2, 5, 6]
assert interweave([1], [4]) == [1, 4]
assert interweave([], []) == []

In [15]:
def sort_special(df):
    # interweave per n_hops, i.e. one 2-hop, one 3-hop, one 4-hop, etc. repeat
    n_hops = df['n_hops'].unique()
    n_hops.sort()
    batches = [df[df['n_hops'] == n_hop].sort_values('id').to_dict(orient='records') for n_hop in n_hops]
    return pd.DataFrame(interweave(*batches))

In [18]:
final_dfs = []
for filepath, df in zip(filepaths, dfs):
    print(filepath.stem)
    print(len(df))
    common_df = df[df['id'].isin(common_ids)].copy()
    print(len(common_df))
    common_df = sort_special(common_df)
    print(len(common_df))
    common_df.to_json(filepath.with_name(f"{filepath.stem}-common.json"), orient="records", indent=2)
    final_dfs.append(common_df)

ccot-auto
168
123
123
cot-auto
150
123
123
cte-auto
174
123
123
direct-auto
159
123
123


In [20]:
# assert that all final dfs have the same id column order
assert all((final_df['id'] == final_dfs[0]['id']).all() for final_df in final_dfs[1:])